In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings('ignore')

class NeuralNetwork:
    
    def __init__(self):
         # Learning Rate
        self.l_rate = 0.001
         # Total iterations
        self.epochs = 2
    
    def gradient_descent(self, x_train_data, y_train_data):
        
        layers = [x_train_data.shape[1], 5, 4, 1]
        z = 0
        w = []
        b = []
        a = [0] * len(layers)
        z = [0] * len(layers)
        d_g = [0] * len(layers)
        da = [0] * len(layers)
        dz = [0] * len(layers)
        db = [0] * len(layers)
        dw = [0] * len(layers)
        
        a[0] = x_train_data.T # 12288x209
        m = x_train_data.shape[0]
        n = x_train_data.shape[1]

        for j in range(1, len(layers)):
            
            w.append(np.random.rand(layers[j], layers[j-1]) * 0.01 ) # 5x12288
            b.append(np.zeros(((layers[j], 1)) ))# 5x1
        for e in range(self.epochs):
            for i in range(len(layers) - 1):  
                print("Layer : ", i)
                ########## Forward Propagation ###########
                print("w ", i, "= ", w[i].shape)
                print("a ", i, "= ", a[i].shape)
                z[i] = np.dot(w[i], a[i]) + b[i] # 5x209
                print("z ", i, "= ", z[i].shape)
                a[i+1] = 1 / (1 + np.exp(-z[i])) # 5x209
                print("a ", i+1,"= ", a[i+1].shape)
        
                ######### Backward Propagation ###########

                # Derivative of sigmoid function
                # d_g = a * (1 - a)
                d_g[i] = (1 / (1 + np.exp(-z[i]))) * (1 - (1 / (1 + np.exp(-z[i])))) # 5x209
                print("dg ", i, "= ", d_g[i].shape)
                
                  # Loss Funtion
                # da = -(y/a) + ((1-y)/(1-a))
                da[i] = (-(y_train_data.T / a[i+1]) + ((1 - y_train_data.T) / (1 - a[i+1]))) # 5x209
                print("da ", i, "= ", da[i].shape)
                
                # dz = da * d_g
                dz[i] = (da[i] * d_g[i]) # 5x209
                print("dz ", i, "= ", dz[i].shape)
                
                #dw = (dz * a) / m
                dw[i] = np.dot(dz[i], a[i].T)# 5x12288
                print("dw ", i, "= ", dw[i].shape)
                # db = np.sum(dz)
                db[i] = np.sum(dz[i], axis=1, keepdims=True) / m # 209x1
                print("db ", i, "= ", db[i].shape)
                # w = w - alpha * dw
                # b = b - alpha * db
                w[i] =  w[i] - np.dot(self.l_rate, dw[i]) # 209x12288
                b[i] = b[i] - np.dot(self.l_rate, db[i]) # 209x1
                print("w ", i, "= ", w[i].shape)
                print("b ", i, "= ", b[i].shape)

        return [w, b]

    def prediction(self, parameters, x_test_data):
        layers = [x_test_data.shape[1], 50, 40, 50, 30, 1]
        a = [0] * 6
        a[0] = x_test_data.T
        z = [0] * 6
        a = [0] * 6
        for i in range(4):  
            z[i] = np.dot(parameters[0][i], a[i]) + parameters[1][i]
            a[i+1] = 1 / (1 + np.exp(-z[i])) # 5x209
        return a[-1]
    
    def categorical_data_handling(self, df):
        categorical_data = []
        for name in list(df.columns):
            if is_numeric_dtype(df[name]):
                pass
            else:
                categorical_data.append(name)
        return pd.get_dummies(df, columns=categorical_data)

    
def main():   

    # Create a class object
    nn = NeuralNetwork()

    df = pd.read_csv("bank.csv", delimiter = ";")

    df.y.replace(('yes', 'no'), (1, 0), inplace=True)
    
    df = nn.categorical_data_handling(df)


    y_data_set = pd.DataFrame(df.y)
    x_data_set = df.drop(["y"], axis=1)
    
    train_size = int(np.ceil((len(y_data_set) * 80) / 100))
    test_size = int(len(y_data_set) - train_size)

    x_train_data = np.array(x_data_set[:train_size])
    y_train_data = np.array(y_data_set[:train_size])
    x_test_data = np.array(x_data_set[train_size:])
    y_test_data = np.array(y_data_set[train_size:])
    
    print(x_train_data.shape)
    print(x_test_data.shape)

    # parameter stores the value of weights and bais
    parameters = nn.gradient_descent(x_train_data, y_train_data)
    
    # pred stores the predicted values from test data set
    pred_test = nn.prediction(parameters, x_test_data) 
    
     # pred stores the predicted values from train data set
    pred_train= nn.prediction(parameters, x_train_data)  
    
    #It calculates the accuracy
    train_accuracy = (100 - np.mean(np.abs(pred_train - y_train_data)) * 100)
    test_accuracy = (100 - np.mean(np.abs(pred_test - y_test_data)) * 100)

    print("train_accuracy = ", train_accuracy)
    print("test_accuracy = ", test_accuracy)


if __name__ == '__main__':
    main()

(3617, 51)
(904, 51)
Layer :  0
w  0 =  (5, 51)
a  0 =  (51, 3617)
z  0 =  (5, 3617)
a  1 =  (5, 3617)
dg  0 =  (5, 3617)
da  0 =  (5, 3617)
dz  0 =  (5, 3617)
dw  0 =  (5, 51)
db  0 =  (5, 1)
w  0 =  (5, 51)
b  0 =  (5, 1)
Layer :  1
w  1 =  (4, 5)
a  1 =  (5, 3617)
z  1 =  (4, 3617)
a  2 =  (4, 3617)
dg  1 =  (4, 3617)
da  1 =  (4, 3617)
dz  1 =  (4, 3617)
dw  1 =  (4, 5)
db  1 =  (4, 1)
w  1 =  (4, 5)
b  1 =  (4, 1)
Layer :  2
w  2 =  (1, 4)
a  2 =  (4, 3617)
z  2 =  (1, 3617)
a  3 =  (1, 3617)
dg  2 =  (1, 3617)
da  2 =  (1, 3617)
dz  2 =  (1, 3617)
dw  2 =  (1, 4)
db  2 =  (1, 1)
w  2 =  (1, 4)
b  2 =  (1, 1)
Layer :  0
w  0 =  (5, 51)
a  0 =  (51, 3617)
z  0 =  (5, 3617)
a  1 =  (5, 3617)
dg  0 =  (5, 3617)
da  0 =  (5, 3617)
dz  0 =  (5, 3617)
dw  0 =  (5, 51)
db  0 =  (5, 1)
w  0 =  (5, 51)
b  0 =  (5, 1)
Layer :  1
w  1 =  (4, 5)
a  1 =  (5, 3617)
z  1 =  (4, 3617)
a  2 =  (4, 3617)
dg  1 =  (4, 3617)
da  1 =  (4, 3617)
dz  1 =  (4, 3617)
dw  1 =  (4, 5)
db  1 =  (4, 1)
w  1 =

IndexError: list index out of range